### Como utilizar ML com Dados Não Estruturais Textuais?

* Técnicas de NLP preprocessam os dados para filtrarmos o que é mais relevante para a classificação 

![Title](../imgs/nlp-illustration.png)


* A tokenização nos auxilia a dividir strings em palavras
* Essas palavras serão as "features" que serão utilizadas para aprendizado de um modelo de Machine Learning
* Um algoritmo de Machine Learning só entende números (atributos sempre numéricos), sendo assim, após a tokenização, precisamos converter nossas palavras para valores numéricos
* O processo de transformação textual para base numérico denomina-se **bag of words**

![Title](../imgs/1-Bag-of-words.png)

* Técnicas como TF-IDF e CountVectorizer, apesar de eficientes, transformam o dataset em dado tabular
  * Cada coluna é uma palavra
  * Quando a palavra existir em uma dada amostra, substituimos o valor 0 pela frequência em que aquela palavra ocorre no texto
  
  <img src="https://www.researchgate.net/profile/Haider-Al-Khateeb/publication/291950178/figure/fig1/AS:330186932408324@1455734107458/Term-Frequency-Inverse-Document-Frequency-TF-IDF.png" width=800>
  
* A consequencia disso é a perda da relação semântica nas frases
* Perda de contexto das sentenças
* Vocabulário imenso (maldição da dimensionalidade)

### Word Embeddings to rescue!

* Vetor multidimensional de palavras
* Mapa de características com a frequência de palavras e considerações semânticas 
* Baseado em similaridade
* Aprendizado por meio de redes neurais

##### Como fazemos isso?
* Dada uma palavra da frase
* Pegar as palavras vizinhas 
* Rede Neural irá calcular a probabilidade dessa palavra vizinha ser a palavra a ser predita
* Selecionamos as palavras vizinhas definindo uma "janela" ou "kernel"
  * Janela = 2 considere duas palavras ao redor da palavra atual
![Title](../imgs/training_data.png)

* Criamos um corpus com as palavras organizadas de acordo com as janelas em torno do dataset.
* Palavras que aparecerem juntas tem maior probabilidade de terem significado semântico
  * Exemplo: "soviética" provavelmente terá uma semelhança com união
  
 ![Title](../imgs/vetor_multi.png)
 
[Leitura adicional](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/): funcionamento de uma rede neural para embeddings

### Arquiteturas Word Embeddings

* Existem diferentes formas de gerar nossas embeddings, gerando determinadas arquiteturas previamente conhecidos como CBOW e SkipGram. Essas arquiteturas também são denominadas como tipo word2vec (palavra para vetor)

* CBOW: predizer uma palavra baseado nas palavras vizinhas. Mais rápido, funciona melhor nas palavras mais frequentes.
* Skip-Gram: predizer o contexto baseado nas palavras vizinhas. Funciona melhor com datasets menores
<img src="https://leimao.github.io/images/article/2019-08-23-Word2Vec-Classic/word2vec.png">
* Ambas arquiteturas tem uma ambientação semelhante, modificando a forma como a saída e a entrada são organizadas
* Exemplo: Hoje vai fazer sol pela manhã com pancadas de chuva à tarde
* Utilizando CBOW\
![Title](../imgs/cbow.png)
* Utilizando Skip-Gram\
![Title](../imgs/skip_gram.png)

[Leitura adicional](https://www.tensorflow.org/tutorials/text/word2vec): transformando o corpus do CBOW/skip_gram em numérico

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec

# Baixa as listas de stopwords e as tokenizações
nltk.download('stopwords')
nltk.download('punkt')

# Define as stopwords em inglês
sw_english = set(stopwords.words('english'))

# Instância o PorterStemmer
stemmer = PorterStemmer()

# Carrega o conjunto de dados
movies = pd.read_csv('../files/movies.csv', index_col = 0)

# Retira uma amostra do conjunto de dados
movies_sample = movies.sample(frac = 0.1, replace=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Faaeel06/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/Faaeel06/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
movies_sample

,text,label
8756,"I was blown away when I saw ""The Best Years of...",1
23887,The revisionist history -- making the evil Mar...,0
17998,"""The 40 Year Old Virgin"" exists in a world I d...",0
34861,"This movie was very funny, I couldn't stop smi...",1
19491,It was a doubly interesting experience. For so...,1
...,...,...
5296,The original Road House was a classic cheesy 8...,0
13522,"Noting the cast, I recently watched this movie...",0
33354,This is an hilarious movie. One of the very be...,1
23479,"I wasn't so impressed with this film, finding ...",0


In [3]:
# Cria um pipeline
def preprocessing(string):
    ###
    # Deixa apenas elementos alfanuméricos
    string = re.sub(r"[^a-zA-Z0-9]+", ' ', string)
    ###
    # deixa todas as palavras minúsculas
    string = string.lower()
    ###
    # tokenização
    words = word_tokenize(string)
    ###
    # Remove Stopwords
    filtered_words = []
    for w in words:
        if w not in sw_english:
            filtered_words.append(w)
    ###
    # Aplica o Stemming
    stem_words = []
    for w in filtered_words:
        s_words = stemmer.stem(w)
        stem_words.append(s_words)
    ###
    # Retorna a lista de palavras pré-processadas
    return stem_words

# Aplica o preprocessing nas críticas de filmes
movies_sample["filtered_words"] = movies_sample['text'].apply(lambda x: preprocessing(x))

In [4]:
movies_sample

,text,label,filtered_words
8756,"I was blown away when I saw ""The Best Years of...",1,"[blown, away, saw, best, year, live, act, scri..."
23887,The revisionist history -- making the evil Mar...,0,"[revisionist, histori, make, evil, marqui, de,..."
17998,"""The 40 Year Old Virgin"" exists in a world I d...",0,"[40, year, old, virgin, exist, world, understa..."
34861,"This movie was very funny, I couldn't stop smi...",1,"[movi, funni, stop, smile, watch, alreadi, wat..."
19491,It was a doubly interesting experience. For so...,1,"[doubli, interest, experi, reason, greatest, s..."
...,...,...,...
5296,The original Road House was a classic cheesy 8...,0,"[origin, road, hous, classic, cheesi, 80, movi..."
13522,"Noting the cast, I recently watched this movie...",0,"[note, cast, recent, watch, movi, tcm, hope, a..."
33354,This is an hilarious movie. One of the very be...,1,"[hilari, movi, one, best, thing, qualiti, perf..."
23479,"I wasn't so impressed with this film, finding ...",0,"[impress, film, find, quit, tiresom, plain, pl..."


In [6]:
model = Word2Vec(movies_sample['filtered_words'])
model = Word2Vec(
        sentences = movies_sample['filtered_words'],
        vector_size= 100,
        min_count= 1,
        workers=8,
        window=5,
        sg=1, #### 1 para skip-gram, 0 para CBOW
)

In [7]:
words = list(model.wv.index_to_key)

In [8]:
print("Primeiras 10 palavras:", words[:10], "\n\n")
print("Palavras similares a time: \n", model.wv.most_similar('time'), "\n\n")
print('Vetor para time:', model.wv.get_vector('time', norm=True), '\n\n')

Primeiras 10 palavras: ['br', 'movi', 'film', 'one', 'like', 'good', 'time', 'make', 'get', 'see'] 


Palavras similares a time: 
 [('celluloid', 0.7939850091934204), ('preciou', 0.7900879383087158), ('split', 0.7758944034576416), ('afterward', 0.7690925598144531), ('thru', 0.7686951160430908), ('nearli', 0.7666786909103394), ('repeat', 0.7666296362876892), ('id', 0.7649783492088318), ('opportun', 0.7625721096992493), ('tomorrow', 0.7605178952217102)] 


Vetor para time: [ 0.02527022  0.04148626  0.05303635  0.07578712  0.00516384 -0.19452904
  0.11483211  0.17773536 -0.04306247 -0.15226576  0.00239717 -0.17972405
 -0.10817587  0.17709993 -0.05973883  0.01985734  0.08162588  0.02268315
 -0.17652161 -0.26361853  0.11976416 -0.03794185  0.07819799 -0.02744842
  0.02710967  0.08144224 -0.12959158 -0.01833709 -0.14740601  0.04021579
  0.13443269  0.12660268  0.05430776 -0.03120637 -0.06164306  0.01505889
  0.02480303 -0.09145157 -0.19220999 -0.07239845  0.01723683 -0.0270832
  0.0260359  -

In [9]:
x = model.wv[words]
x

array([[-1.91651925e-01,  1.36527941e-01,  1.95569545e-01, ...,
        -1.15626596e-01,  2.05697529e-02,  2.71272480e-01],
       [-5.02077758e-01,  8.72410834e-02,  9.16646868e-02, ...,
        -2.41754025e-01, -1.79974675e-01,  7.60941580e-02],
       [-2.98758149e-01,  2.72341847e-01,  6.98877848e-04, ...,
        -1.81508362e-01, -2.36868218e-01,  1.29560307e-01],
       ...,
       [-1.00133777e-01,  1.35118052e-01,  1.05701238e-01, ...,
        -6.13865927e-02,  7.07681244e-03,  2.64676902e-02],
       [-7.79602155e-02,  1.13227785e-01,  9.12993103e-02, ...,
        -3.19390856e-02,  4.47358529e-04, -6.22297055e-04],
       [-5.76751120e-02,  9.26765278e-02,  6.27294704e-02, ...,
        -3.61044928e-02, -1.37572829e-03,  1.18741812e-02]], dtype=float32)

In [10]:
model.wv.similarity('time','second')

0.684262

### Treinamento de ML

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(movies_sample["filtered_words"],
                                                    movies_sample['label'],
                                                    test_size=0.3,
                                                    random_state=42)

In [12]:
model = Word2Vec(X_train)

In [13]:
words = list(model.wv.index_to_key)
X_train_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_train])
X_test_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_test])

/tmp/ipykernel_18831/4006831418.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_train])
/tmp/ipykernel_18831/4006831418.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([model.wv[i] for i in ls if i in words]) for ls in X_test])


In [14]:
X_test_vect

array([array([[-0.6228106 ,  0.59790635,  0.2813161 , ..., -1.0105655 ,
               -0.506629  ,  0.29375982],
              [-0.15514997,  0.25937256,  0.01537842, ..., -0.20945041,
               -0.00947169, -0.0196006 ],
              [-0.15151815,  0.6922427 , -0.07741792, ..., -0.25776216,
               -0.24856935,  0.04601016],
              ...,
              [-0.44970167,  0.05530951,  0.22182508, ..., -0.29104635,
               -0.69238734,  0.06700387],
              [-0.41790688, -0.01518096,  0.24296077, ..., -0.5936734 ,
               -0.18817015, -0.01835886],
              [-0.7698827 , -0.17955036,  0.14720777, ..., -0.8593956 ,
               -0.7365144 , -0.14187057]], dtype=float32)              ,
       array([[-0.3166941 ,  0.3639355 ,  0.13409203, ..., -0.53619903,
               -0.04459017,  0.01239145],
              [-0.6228106 ,  0.59790635,  0.2813161 , ..., -1.0105655 ,
               -0.506629  ,  0.29375982],
              [-0.12308968,  0.1988001

In [ ]:
model = RandomForestClassifier()
model.fit(X_train_vect, y_train)
y_pred = model.predict(X_test_vect)

print(classification_report(y_test, y_pred))